In [2]:
from fastapi import FastAPI
import uvicorn
import nest_asyncio
import requests

In [17]:
app = FastAPI()

fake_items_db = [
    {"item_name": "Apple", "price": 10, "category": "fruits"},
    {"item_name": "Banana", "price": 5, "category": "fruits"},
    {"item_name": "Carrot", "price": 3, "category": "vegetables"},
    {"item_name": "Broccoli", "price": 4, "category": "vegetables"},
]


@app.get("/furits/data/")
async def read_item(filter_: str = None,by : str = None):

    items = fake_items_db.copy()
    
    if filter_:
        items = [i for i in items if i.get(filter_) == by]
    
    return items


if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app,port=4444)

INFO:     Started server process [5660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:54160 - "GET /furits/data/?filter_=category&by=fruits HTTP/1.1" 200 OK
INFO:     127.0.0.1:54161 - "GET /furits/data/?filter_=category&by=vegetables HTTP/1.1" 200 OK
INFO:     127.0.0.1:54162 - "GET /furits/data/?filter_=item_name&by=Carrot HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5660]
